Importujemy biblioteki i wczytujemy pliki CSV

In [4]:
from pyspark.ml.feature import StringIndexer
from pyspark.sql import SparkSession
from sklearn.preprocessing import LabelEncoder
from LogParser import  LogParser
import pandas as pd
import re
import pickle

DIRECTORY = 'data'

spark = SparkSession.builder.appName("Preprocess data").getOrCreate()
ddos_tf_df = spark.read.format("csv").option("header", "true").load(DIRECTORY + "/ddos-tcp-syn-flood.csv")
normal_tf_df = spark.read.format("csv").option("header", "true").load(DIRECTORY + "/normal-traffic.csv")
port_scan_tf_df = spark.read.format("csv").option("header", "true").load(DIRECTORY+ "/port-scanning.csv")

data_frames = {
    "normal-traffic": normal_tf_df,
    "port-scanning": port_scan_tf_df,
    "ddos-tcp-syn-flood": ddos_tf_df,
}

Wybieramy sobie kolumny zawierające istotne iformacje. Można dodać więcej ale wtedy trzeba pamiętać o noramlizacji w kolejnej komórce.

In [2]:
selected_columns = [
    "frame-time",
    "ip-src_host",
    "ip-dst_host",
    "tcp-connection-syn",
    "tcp-connection-synack",
    "tcp-flags_index",
    "tcp-len",
    "tcp-seq",
    "tcp-dstport",
    "tcp-srcport",
    "Attack_type"
]

Iterujemy po wczytanych ramkach, zamieniamy nazwy kolumn na takie bez kropek i normalizujemy/kodujemy nieliczbowe kolumny (oprócz timestampów, ta kolumna jest modyfikowana później). Odchudzone dane zapisujemy do katalogu `preprocessed_data`

In [3]:
for df_name, df in data_frames.items():

    for col_name in df.columns:
        new_col_name = re.sub(r'\.', '-', col_name)
        df = df.withColumnRenamed(col_name, new_col_name)
    
    tcp_flags_indexer = StringIndexer(inputCol="tcp-flags", outputCol="tcp-flags_index")
    indexed_df = tcp_flags_indexer.fit(df).transform(df)
    
    indexed_df = indexed_df.select([c for c in indexed_df.columns if c in selected_columns])
    pandas_df = indexed_df.toPandas()

    all_ips = pd.concat([pandas_df["ip-src_host"], pandas_df["ip-dst_host"]]).unique()
    label_encoder = LabelEncoder()
    label_encoder.fit(all_ips)
    pandas_df["ip-src_host"] = label_encoder.transform(pandas_df["ip-src_host"])
    pandas_df["ip-dst_host"] = label_encoder.transform(pandas_df["ip-dst_host"])

    pandas_df.to_csv(f'{DIRECTORY}/{df_name}-preprocessed.csv', index=False)

Wczytujemy zapisane pliki csv i tworzymy próbki z danymi, gdzie jedna próbka X to lista zawierająca kolejne 32 logi gdzie od każdego timestampa został odjęty timestamp pierwszego loga z listy (w ten sposób timestampy są niewielkimi wartościami liczbowymi a jednocześnie przechowują informację o odległości pomiędzy kolejnymi logami), a próbka Y to pojedynczy numer określający typ ataku/ruchu normalnego dla zagregowanych logów.

In [7]:
encoded_attacks = {
    "normal-traffic": 0,
    "port-scanning": 1,
    "ddos-tcp-syn-flood": 2
}
x_data = []
y_data = []
DIRECTORY = 'generated_data'
for df_name in data_frames.keys():
    df = pd.read_csv(f'{DIRECTORY}/{df_name}-preprocessed.csv', parse_dates=['frame-time'])
    log_series = LogParser.logs_to_series(df, 32)
    x_data.extend(log_series)
    y_data.extend([encoded_attacks.get(df_name)] * len(log_series))

/home/kuba/Studies/Semester_2/MSIwC/aiics-project/LogParser.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bucket['frame-time'] = bucket['frame-time'] - bucket['frame-time'].iloc[0]
/home/kuba/Studies/Semester_2/MSIwC/aiics-project/LogParser.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bucket['frame-time'] = bucket['frame-time'] - bucket['frame-time'].iloc[0]
/home/kuba/Studies/Semester_2/MSIwC/aiics-project/LogParser.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a s

Przed treningem modeli należy jeszcze pomieszać próbki z danymi oraz podzielić na zbiory treningowe i testowe. W sumie dobrze by też było dodać jakiś padding dla przypadków gdzie jednak próbka nie ma 32 logów.

In [8]:
output_path = f'{DIRECTORY}/processed_data.pkl'
print("Writing log series into ", output_path)
with open(output_path, 'wb') as f:
    pickle.dump((x_data, y_data), f)

Writing log series into  generated_data/processed_data.pkl
